In [20]:
import numpy as np
import subprocess
import shlex
from glob import glob

files = sorted(glob("../../tmp/output_*.npz"))

In [1]:
import numpy as np
import subprocess, shlex

data = np.load('../../tmp/output_000000.npz')
frames = data['frames']           # shape (N, H, W, 4) BGRA
n, H, W, C = frames.shape
assert C == 4

# convert to true RGB
rgb = frames[..., :3][..., ::-1]  # (N, H, W, 3)

cmd = (
    f"ffmpeg -y "
    f"-f rawvideo -vcodec rawvideo "
    f"-pix_fmt rgb24 -s {W}x{H} -r 10 -i - "
    f"-an -c:v libx264 -pix_fmt yuv420p output.mp4"
)
proc = subprocess.Popen(shlex.split(cmd), stdin=subprocess.PIPE)
proc.stdin.write(rgb.tobytes())
proc.stdin.close()
ret = proc.wait()
if ret: raise RuntimeError("ffmpeg failed")
print("✅ output.mp4")


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

✅ output.mp4


[out#0/mp4 @ 0x14ce2f920] video:3193KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.073026%
frame=  128 fps= 44 q=-1.0 Lsize=    3196KiB time=00:00:12.60 bitrate=2077.8kbits/s speed=4.38x    
[libx264 @ 0x14ce30ad0] frame I:5     Avg QP:10.83  size:353983
[libx264 @ 0x14ce30ad0] frame P:39    Avg QP:19.30  size: 24036
[libx264 @ 0x14ce30ad0] frame B:84    Avg QP:17.32  size:  6691
[libx264 @ 0x14ce30ad0] consecutive B-frames: 10.2%  4.7%  7.0% 78.1%
[libx264 @ 0x14ce30ad0] mb I  I16..4: 51.7% 27.0% 21.3%
[libx264 @ 0x14ce30ad0] mb P  I16..4:  0.7%  1.6%  1.8%  P16..4:  1.4%  0.2%  0.1%  0.0%  0.0%    skip:94.1%
[libx264 @ 0x14ce30ad0] mb B  I16..4:  0.1%  0.1%  0.5%  B16..8:  2.3%  0.1%  0.0%  direct: 0.0%  skip:96.7%  L0:51.0% L1:48.4% BI: 0.6%
[libx264 @ 0x14ce30ad0] 8x8 transform intra:28.6% inter:26.5%
[libx264 @ 0x14ce30ad0] coded y,uvDC,uvAC intra: 22.1% 27.7% 25.6% inter: 0.2% 0.2% 0.1%
[libx264 @ 0x14ce30ad0] i16 v,h,dc,p: 82% 17%  1%  0%


In [ ]:
ffmpeg -f avfoundation -framerate 30 \
    -use_wallclock_as_timestamps 1 -i "1:none" \
    -c:v libx264 -g 15 -c:a aac -preset veryfast -crf 17 \
    -copyts -muxdelay 0 \
    -segment_time 10 -f segment \
    -r 30 tmp/video_%03d.mp4

In [21]:
def load_npz(fname):
    data = np.load(fname)
    frames = data['frames']
    return frames

In [22]:
cmd = (
    "ffmpeg -y "
    "-f rawvideo -vcodec rawvideo "
    "-pix_fmt bgra -s {w}x{h} -r 10 -i - "
    "-an -c:v libx264 -pix_fmt yuv420p output.mp4"
)

In [23]:
from concurrent.futures import ThreadPoolExecutor
pool = ThreadPoolExecutor(max_workers=1)

In [24]:
current_idx = 0
current = load_npz(files[current_idx])
H, W = current.shape[1:3]

In [ ]:
proc = subprocess.Popen(
    shlex.split(cmd.format(w=W, h=H)),
    stdin=subprocess.PIPE
)

future = None
if current_idx + 1 < len(files):
    future = pool.submit(load_npz, files[current_idx+1])

while True:
    # 5a) send all frames from current buffer
    proc.stdin.write(current.tobytes())

    # 5b) move on
    current_idx += 1
    if future is None:
        break   # no more files
    # get the preloaded data
    next_buf = future.result()
    # kick off preload of the next one
    future = None
    if current_idx+1 < len(files):
        future = pool.submit(load_npz, files[current_idx+1])
    # swap in
    current = next_buf

proc.stdin.close()
ret = proc.wait()
if ret != 0:
    raise RuntimeError(f"ffmpeg exited {ret}")
print("✅ output.mp4 created!")

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

BadZipFile: File is not a zip file

In [6]:
import av

In [ ]:
# VIDEO_PATH = '../../../tmp/2025-06-10_231949_ABTR8/screen_capture_000000.mp4'
VIDEO_PATH = "../../../tmp/2025-06-10_231949_ABTR8/screen_capture_000050.mp4"
container = av.open(VIDEO_PATH)
video_stream = container.streams.video[0]

FileNotFoundError: [Errno 2] No such file or directory: 'repos/actioncollector/tmp/2025-06-10_231949_ABTR8/screen_capture_000050.mp4'

In [26]:
import json
ACTIONS_PATH = [
    f'../../../tmp/2025-06-10_231949_ABTR8/action_capture_{i:06d}.jsonl'
    for i in range(0, 13)
]

# Load actions (one JSON record per line)
actions = []
for path in ACTIONS_PATH:
    with open(path, 'r') as f:
        for line in f:
            rec = json.loads(line)
            act = rec['action']
            if act["action"] == "mouse_move":
                # timestamp assumed in milliseconds
                actions.append({'timestamp': rec['timestamp'], 'x': act['x']*2, 'y': act['y']*2})

In [29]:
actions.sort(key=lambda a: a['timestamp'])
timestamps = [a['timestamp'] for a in actions]

In [31]:
print(actions[-1])

{'timestamp': 1749612565130.947, 'x': 364, 'y': 702}


In [ ]:
import datetime
from PIL import ImageDraw
import bisect
from IPython.display import display, clear_output

previous_time = None
for i, frame in enumerate(container.decode(video=0)):
    # frame.pts is the integer PTS in stream time_base units
    pts = frame.pts
    # stream.time_base is a Fraction, so pts * time_base gives seconds
    seconds = float(pts * video_stream.time_base)
    # seconds is a utc timestamp in seconds
    actual_frame_time = datetime.datetime.fromtimestamp(seconds)

    clear_output(wait=True)

    print(f"Frame #{i}: PTS={pts:7d}  ({seconds:.3f} s); {actual_frame_time.isoformat()}")
    if previous_time is not None:
        delta = seconds - previous_time
        print(f"  Delta: {delta:.3f} s")
    else:
        print("  First frame")

    prev_idx = bisect.bisect_left(timestamps, seconds * 1e3)
    next_idx = bisect.bisect_right(timestamps, seconds * 1e3)
    idx = bisect.bisect_left(timestamps, seconds * 1e3) + 1

    low = actions[idx-1] if idx > 0 else None
    high = actions[idx] if idx < len(timestamps) else None
    if low is None and high is not None:
        low = high
    if high is None and low is not None:
        high = low

    # if idx == 0:
    #     prev_idx, next_idx = 0, 1 if len(actions) > 1 else 0
    # elif idx >= len(actions):
    #     prev_idx, next_idx = len(actions) - 2 if len(actions) > 1 else 0, len(actions) - 1
    # else:
    #     prev_idx, next_idx = idx, idx + 1

    action_prev = low
    action_prev_time = action_prev['timestamp']/1000.0
    action_next = high
    action_next_time = action_next['timestamp']/1000.0
    print(f"prev action time: {action_prev_time:.3f}s, video time: {seconds:.3f}s, diff: {action_prev_time - seconds:.3f}s")
    print(f"next action time: {action_next_time:.3f}s, video time: {seconds:.3f}s, diff: {action_next_time - seconds:.3f}s")

    previous_time = seconds

    # lerp by the time difference
    mid_x = action_prev['x'] + (action_next['x'] - action_prev['x']) * (seconds - action_prev_time) / (action_next_time - action_prev_time)
    mid_y = action_prev['y'] + (action_next['y'] - action_prev['y']) * (seconds - action_prev_time) / (action_next_time - action_prev_time)
    # niave guess
    # mid_x = (action_prev['x'] + action_next['x']) / 2
    # mid_y = (action_prev['y'] + action_next['y']) / 2

    img = frame.to_image().convert('RGB')
    draw = ImageDraw.Draw(img)
    r = 20  # radius in pixels
    print(img.size, mid_x, mid_y)
    draw.ellipse((mid_x - r, mid_y - r, mid_x + r, mid_y + r), outline='red', width=3)

    # Display the annotated frame
    display(img)

    # if i >= 100:
    #     input("Next frame?")
    # if i >= 10:
    #     break
    

In [25]:
output = av.open('output.mp4', mode='w')
out_stream = output.add_stream('libx264', rate=video_stream.average_rate)
out_stream.width = video_stream.width
out_stream.height = video_stream.height
out_stream.pix_fmt = 'yuv420p'

previous_time = None

container.seek(0, any_frame=True)  # Reset to the start of the video
for i, frame in enumerate(container.decode(video=0)):
    # Compute frame time
    pts = frame.pts
    one_frame_time = float(512 * video_stream.time_base)
    seconds = float(pts * video_stream.time_base) - one_frame_time

    # Find surrounding actions
    idx = bisect.bisect_left(timestamps, seconds * 1000)
    low_idx = max(idx - 1, 0)
    high_idx = min(idx, len(actions) - 1)

    action_prev = actions[low_idx]
    action_next = actions[high_idx]
    action_prev_time = action_prev['timestamp'] / 1000.0
    action_next_time = action_next['timestamp'] / 1000.0

    print(f"Frame #{i}: PTS={pts:7d}  ({seconds:.3f} s); "
          f"Prev action at {action_prev_time:.3f}s, Next action at {action_next_time:.3f}s")

    # Linear interpolation (avoid division by zero)
    if action_next_time != action_prev_time:
        t = (seconds - action_prev_time) / (action_next_time - action_prev_time)
    else:
        t = 0.0
    mid_x = action_prev['x'] + (action_next['x'] - action_prev['x']) * t
    mid_y = action_prev['y'] + (action_next['y'] - action_prev['y']) * t

    # Draw overlay on PIL image
    img = frame.to_image().convert('RGB')
    draw = ImageDraw.Draw(img)
    r = 20
    draw.ellipse((mid_x - r, mid_y - r, mid_x + r, mid_y + r), outline='red', width=3)

    # Convert back to VideoFrame
    annotated = av.VideoFrame.from_image(img)

    # Encode and mux
    for packet in out_stream.encode(annotated):
        output.mux(packet)

# Flush encoder
for packet in out_stream.encode():
    output.mux(packet)

output.close()
print("Finished writing output.mp4")


Frame #0: PTS=26874040159734  (1749611989.533 s); Prev action at 1749611989.530s, Next action at 1749611989.538s
Frame #1: PTS=26874040160246  (1749611989.566 s); Prev action at 1749611989.562s, Next action at 1749611989.569s
Frame #2: PTS=26874040160758  (1749611989.599 s); Prev action at 1749611989.593s, Next action at 1749611989.601s
Frame #3: PTS=26874040161270  (1749611989.633 s); Prev action at 1749611989.633s, Next action at 1749611989.641s
Frame #4: PTS=26874040161782  (1749611989.666 s); Prev action at 1749611989.664s, Next action at 1749611989.672s
Frame #5: PTS=26874040162294  (1749611989.699 s); Prev action at 1749611989.696s, Next action at 1749611989.704s
Frame #6: PTS=26874040162806  (1749611989.733 s); Prev action at 1749611989.729s, Next action at 1749611989.735s
Frame #7: PTS=26874040163318  (1749611989.766 s); Prev action at 1749611989.759s, Next action at 1749611990.044s
Frame #8: PTS=26874040163830  (1749611989.799 s); Prev action at 1749611989.759s, Next action at

0.03333333333333333

In [ ]:
1749611989.699
1749611989.388258
1749611989.699
1749611989.388258
1749611989388.258

1749611989566.0156

In [32]:
(26874040162294-26874040161782)*video_stream.time_base

Fraction(1, 30)

In [14]:
container.duration

10166667